In [155]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [156]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [181]:
df=pd.read_csv(r"C:\Users\ASUS\Downloads\amazonreview.csv")

In [183]:
df=df[['reviews.rating','reviews.text']]

In [184]:
df.columns=['rating','review']
df.head()

,rating,review
0,5.0,This product so far has not disappointed. My c...
1,5.0,great for beginner or experienced person. Boug...
2,5.0,Inexpensive tablet for him to use and learn on...
3,4.0,I've had my Fire HD 8 two weeks now and I love...
4,5.0,I bought this for my grand daughter when she c...


In [185]:
df["rating"] = df["rating"].apply(lambda x: 1 if (x ==5 or x==4) else -1 if (x == 1 or x==2) else 0)
df.head()

,rating,review
0,1,This product so far has not disappointed. My c...
1,1,great for beginner or experienced person. Boug...
2,1,Inexpensive tablet for him to use and learn on...
3,1,I've had my Fire HD 8 two weeks now and I love...
4,1,I bought this for my grand daughter when she c...


Prepocessing

In [186]:
import string
def preprocess(review):
    review=str(review)
    review.lower()
    review=re.sub('^[0-9a-b]','',review)
    review=review.translate(str.maketrans("","",string.punctuation))
    review=re.sub(r"\@\w+/|\#","",review)
    
    review_tokens=word_tokenize(review)
    filtered_words=[word for word in review_tokens if word not in stop_words]
    
    lemma=WordNetLemmatizer()
    lem_words= [lemma.lemmatize(w, pos='a') for w in filtered_words]
    
    return " ".join(lem_words)


In [187]:
df['review']=df['review'].apply(preprocess)
df.head()

,rating,review
0,1,This product far disappointed My children love...
1,1,great beginner experienced person Bought gift ...
2,1,Inexpensive tablet use learn step NABI He thri...
3,1,Ive Fire HD 8 two weeks I love This tablet gre...
4,1,I bought grand daughter comes visit I set user...


In [188]:
tv = TfidfVectorizer(max_features=2500)
X = tv.fit_transform(df['review'] ).toarray()
y = df['rating'] 

In [189]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [190]:
y_test

1763     1
20879    1
26427    1
8344     1
34279    1
        ..
1322     1
1563     1
789      1
19674    1
1165     1
Name: rating, Length: 6932, dtype: int64

In [191]:
clf = LogisticRegression()

# Train the model on the training data
clf.fit(X_train, y_train)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [192]:

y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Generate a classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.94
              precision    recall  f1-score   support

          -1       0.68      0.17      0.27       161
           0       0.58      0.09      0.15       288
           1       0.94      1.00      0.97      6483

    accuracy                           0.94      6932
   macro avg       0.73      0.42      0.46      6932
weighted avg       0.92      0.94      0.92      6932



In [193]:
x=set(y_pred)
x

{-1, 0, 1}

In [194]:
NB_model = MultinomialNB()
NB_model.fit(X_train,y_train)
NB_model.score(X_test,y_test)

0.9352279284477785

In [195]:
y_pred=NB_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       161
           0       0.50      0.01      0.02       288
           1       0.94      1.00      0.97      6483

    accuracy                           0.94      6932
   macro avg       0.48      0.34      0.33      6932
weighted avg       0.90      0.94      0.90      6932



In [196]:
x=set(y_pred)
x

{-1, 0, 1}

In [197]:
svc = SVC(kernel='linear', C=1.0) 
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Generate a classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.94
              precision    recall  f1-score   support

          -1       0.66      0.14      0.23       161
           0       0.00      0.00      0.00       288
           1       0.94      1.00      0.97      6483

    accuracy                           0.94      6932
   macro avg       0.53      0.38      0.40      6932
weighted avg       0.89      0.94      0.91      6932



C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
